## Packages

Install and import the TF-Ranking library:

In [1]:
# !pip install -q tensorflow-ranking
# !pip install -q --upgrade tensorflow-datasets

In [2]:
%load_ext autoreload
%autoreload 2
import sys

import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from utils import *
from typing import Dict, Tuple

import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_ranking as tfr

import polars as pl
import pandas as pd
import numpy as np

import copy
from src.eval import model_eval

import joblib
import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from gensim.similarities.annoy import AnnoyIndexer


from annoy import AnnoyIndex
import polars as pl
import implicit
from src.eval import model_eval

import scipy.sparse as sps
from utils import str2list
from src.config import raw_data_session_id_dir, candidate_file_name
from lightgbm import LGBMRanker
from lightgbm import early_stopping
from utils import *
from src.case_analysis import show_single_case
# try github 


2023-06-12 05:25:30.371170: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


## Config 

In [3]:
debug = True

task = 'task2'

topn = 100
version = 'v3'
# # target locales: locales needed for task1
if task == 'task1':
    target_locals = ["DE", 'JP', 'UK']
elif task == 'task2':
    target_locals = ['ES', 'FR', 'IT']
else:
    assert 1 == 2
    
    
eval_frac = 0.1


ranker_train_data_dir = f'../data/rank_train_data_{task}_{version}'

rank_model_version = f'{task}_rank_tf_{version}'

epochs = 100

if debug:
    epochs = 4


In [4]:

if debug:
    SAMPLE_NUM = 3000
else:
    SAMPLE_NUM = -1

candidate_path = '../data/candidates/'
model_dir = '../model_training'


w2v_model_version = f'w2v_{task}'
nic_model_version = f'nic_{task}'

rank_model_dir = os.path.join(model_dir, rank_model_version)
model_for_eval = True
# w2v_topn=100
# nic_topn=100
# PREDS_PER_SESSION = 100

# num_tree = 100

# submit_file = f'submission_{task}_ALS.parquet'
num_tree = 100
w2v_model_dir = os.path.join(model_dir, w2v_model_version)
w2v_model_file = os.path.join(w2v_model_dir, f"{model_for_eval}.model")
annoy_index_file = os.path.join(w2v_model_dir, f"{str(num_tree)}_{model_for_eval}.index")


sub_file = f'../data/sub_files/{rank_model_version}.parque'
eval_sub_file = f'../data/sub_files/eval_{rank_model_version}.parque'

In [5]:
# cols_needed = ['session_id', 'next_item_prediction', 'target']


user_col = 'session_id'
item_col = 'next_item_prediction'

# target_session_id = [3273855,
#  3273726,
#  3272767,
#  3273728,
#  3273277,
#  3273790,
#  3273536,
#  3273022,
#  3273216,
#  3273663,
#  3273727,
#  3272766,
#  3273343,
#  3273664,
#  3273599,
#  3273342,
#  3273598,
#  3273919,
#  3273344,
#  3273470]

In [6]:

if debug:
    
    eval_pl = (pl.scan_parquet(os.path.join(base_dir,
                                           raw_data_session_id_dir,
                                           'sessions_eval.parquet')).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))
                  .head(SAMPLE_NUM)
              )

    test_pl = (pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, f'sessions_test_{task}.parquet'), ).with_columns(pl.col('prev_items').apply(str2list))
               .head(SAMPLE_NUM)
              )
    # test4task3_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task3.parquet'), ).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list)).head(SAMPLE_NUM)

    train_candidates = (pl.scan_parquet(os.path.join(ranker_train_data_dir, 'train/' '*.parquet'), ).head(SAMPLE_NUM)
                           .with_columns(pl.col('session_id').cast(pl.Int32).cast(pl.Utf8))
                       )
    eval_candidates = (pl.scan_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'), ).head(SAMPLE_NUM)
                          .with_columns(pl.col('session_id').cast(pl.Int32).cast(pl.Utf8))
                      )
    test_candidates = (pl.scan_parquet(os.path.join(ranker_train_data_dir, 'test.parquet'), ).head(SAMPLE_NUM)
                          .with_columns(pl.col('session_id').cast(pl.Int32).cast(pl.Utf8))
                      )
    # test4task3_candidates = (pl.scan_parquet(os.path.join(ranker_train_data_dir, 'test4task3.parquet'), ).head(SAMPLE_NUM)
    #                             .with_columns(pl.col('session_id').cast(pl.Int32).cast(pl.Utf8))
    #                         )
else:
    eval_pl = (pl.scan_parquet(os.path.join(base_dir,
                                           raw_data_session_id_dir,
                                           'sessions_eval.parquet')).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))
                  # .head(SAMPLE_NUM)
              )

    test_pl = (pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, f'sessions_test_{task}.parquet'), ).with_columns(pl.col('prev_items').apply(str2list))
               # .head(SAMPLE_NUM)
              )
    # test4task3_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task3.parquet'), ).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list)).head(SAMPLE_NUM)

    train_candidates = (pl.scan_parquet(os.path.join(ranker_train_data_dir, 'train/' '*.parquet'), )
                            # .head(SAMPLE_NUM)
                           .with_columns(pl.col('session_id').cast(pl.Int32).cast(pl.Utf8))
                       )
    eval_candidates = (pl.scan_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'), )
                       # .head(SAMPLE_NUM)
                          .with_columns(pl.col('session_id').cast(pl.Int32).cast(pl.Utf8))
                      )
    test_candidates = (pl.scan_parquet(os.path.join(ranker_train_data_dir, 'test.parquet'), )
                       # .head(SAMPLE_NUM)
                          .with_columns(pl.col('session_id').cast(pl.Int32).cast(pl.Utf8))
                      )
    # test4task3_candidates = (pl.scan_parquet(os.path.join(ranker_train_data_dir, 'test4task3.parquet'), ).head(SAMPLE_NUM)
    #                             .with_columns(pl.col('session_id').cast(pl.Int32).cast(pl.Utf8))
    #                         )


## Function 

In [16]:
def get_tensor_data(target_pl,  feature_cols=[]):
    train_df = target_pl.collect().to_pandas()
    print(f"train: {train_df.shape}")
    print(train_df['target'].value_counts())
    cols_needed = ['user_rating', 'user_id', 'movie_title']+feature_cols
    train_df = train_df.rename(columns={

        'session_id': 'user_id'
        , 'next_item_prediction': 'movie_title'
        , 'target': 'user_rating'
    })
    ratings = tf.data.Dataset.from_tensor_slices(dict(train_df[cols_needed]))
    return ratings


def get_dataset(ratings, user_ids_vocabulary):
    key_func = lambda x: user_ids_vocabulary(x["user_id"])
    reduce_func = lambda key, dataset: dataset.batch(100)
    ds_train = ratings.group_by_window(
        key_func=key_func, reduce_func=reduce_func, window_size=100)

    def _features_and_labels(
        x: Dict[str, tf.Tensor]) -> Tuple[Dict[str, tf.Tensor], tf.Tensor]:
      labels = x.pop("user_rating")
      return x, labels

    ds_train = ds_train.map(_features_and_labels)

    ds_train = ds_train.apply(
        tf.data.experimental.dense_to_ragged_batch(batch_size=32))
    return ds_train

## Data process 

In [8]:
product_df = (
    pl.scan_parquet('../data/raw_data_session_id/products_train.parquet')
).filter(pl.col('locale').is_in(target_locals)).collect().to_pandas()
print(f"product: {product_df.shape}")

product_df = product_df.rename(columns={
    
    'id': 'movie_title'

})
movies = tf.data.Dataset.from_tensor_slices(dict(product_df[['movie_title']]))

product: (137541, 12)


2023-06-12 05:25:50.608596: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


### More features 

In [9]:
session_features = pl.scan_parquet('../data/session_item_features/session_features.parquet')
item_features = pl.scan_parquet('../data/session_item_features/item_features.parquet')
product_pl = (pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 
                                          'products_train.parquet'))
                  # .with_columns(
                  #     pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
                  #       .then(2)
                  #       .otherwise(3).alias('locale')
                  # )
             )

def get_additional_feature(data_pl, session_features=session_features, product_pl=product_pl, item_features=item_features):
    session_join_keys = ['session_id']
    session_fea_cols = ['mean_price', 'last_price', 'min_price', 'max_price']
    data_pl = (
        data_pl
         .with_columns(
            pl.col('session_id').cast(pl.Int64)
        )
        .join(session_features.select(session_join_keys+session_fea_cols+['locale']),
                                    how='left', on=session_join_keys, suffix='_str')
            .join(item_features, how='left',
                  left_on='next_item_prediction',
                  right_on='prev_items')
            .join(product_pl.select(['id', 'locale', 'price']), how='left', right_on=['id', 'locale']
                 , left_on=['next_item_prediction', 'locale_str'])
            .with_columns(
              pl.col('price').truediv(pl.col('mean_price')).alias('price_mean_norm')
              , pl.col('price').truediv(pl.col('last_price')).alias('price_last_norm')
              , pl.col('price').truediv(pl.col('min_price')).alias('price_min_norm')
              , pl.col('price').truediv(pl.col('max_price')).alias('price_max_norm')
              # , pl.when(pl.col('min_price')==pl.col('max_price')).then(1).otherwise((pl.col('price')-pl.col('min_price'))/(pl.col('min_price')-pl.col('max_price'))).alias('price_norm')
            )
            .select(
                pl.all().exclude(session_fea_cols+['locale_str', 'price'])
            )

    )
    return data_pl

In [10]:
target_train_candidates = get_additional_feature(data_pl=train_candidates)

In [14]:
if debug:
    print(target_train_candidates.head().collect())
    print(target_train_candidates.schema)

shape: (5, 66)
┌──────────┬────────────┬──────────┬───────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ session_ ┆ next_item_ ┆ w2v_weig ┆ whether_w ┆ … ┆ price_mean ┆ price_last ┆ price_min_ ┆ price_max_ │
│ id       ┆ prediction ┆ ht       ┆ 2v        ┆   ┆ _norm      ┆ _norm      ┆ norm       ┆ norm       │
│ ---      ┆ ---        ┆ ---      ┆ ---       ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│ i64      ┆ str        ┆ f64      ┆ i64       ┆   ┆ f64        ┆ f64        ┆ f64        ┆ f64        │
╞══════════╪════════════╪══════════╪═══════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ 3272766  ┆ B09C5VHGBZ ┆ 0.0      ┆ 0         ┆ … ┆ 0.633813   ┆ 0.5        ┆ 1.0        ┆ 0.5        │
│ 3272766  ┆ B09FZ8ZZB2 ┆ 1.0      ┆ 1         ┆ … ┆ null       ┆ null       ┆ null       ┆ null       │
│ 3272766  ┆ B09D3D8YR6 ┆ 0.21     ┆ 1         ┆ … ┆ 0.704315   ┆ 0.555617   ┆ 1.111235   ┆ 0.555617   │
│ 3272766  ┆ B0B6W2CXN7 ┆ 0.01     ┆ 1  

In [15]:

target_train_candidates = train_candidates
train_session_id = target_train_candidates.select(pl.col('session_id')).unique().collect()

session_num = train_session_id.shape[0]
random_pl = train_session_id.with_columns(
        pl.Series(name='random', values=np.random.uniform(size=session_num))
)
train_candidates = target_train_candidates.join(random_pl.lazy(), how='left', on='session_id')

sampled_train_candidates = train_candidates.filter(pl.col('random')>eval_frac).sort('session_id', descending=False).collect()
sampled_eval_candidates = train_candidates.filter(pl.col('random')<=eval_frac).sort('session_id', descending=False).collect()

In [17]:

feature_cols = ['next_item_normalized_weight']

ratings = get_tensor_data(target_pl=train_candidates, feature_cols=feature_cols)
sampled_train_dataset = get_tensor_data(target_pl=sampled_train_candidates.lazy(), feature_cols=feature_cols)
sampled_eval_dataset = get_tensor_data(target_pl=sampled_eval_candidates.lazy(), feature_cols=feature_cols)


train: (3000, 59)
target
0.0    2979
1.0      21
Name: count, dtype: int64
train: (2873, 59)
target
0.0    2853
1.0      20
Name: count, dtype: int64
train: (127, 59)
target
0.0    126
1.0      1
Name: count, dtype: int64


In [20]:
if debug:
    for row in sampled_eval_dataset.take(3):
        print(row)

{'user_rating': <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'3273343'>, 'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'B0068BBBIU'>, 'next_item_normalized_weight': <tf.Tensor: shape=(), dtype=float64, numpy=0.0>}
{'user_rating': <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'3273343'>, 'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'B000CST6NM'>, 'next_item_normalized_weight': <tf.Tensor: shape=(), dtype=float64, numpy=0.0>}
{'user_rating': <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'3273343'>, 'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'B09S6SXHS9'>, 'next_item_normalized_weight': <tf.Tensor: shape=(), dtype=float64, numpy=0.0>}


2023-06-12 05:31:13.582359: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype double and shape [127]
	 [[{{node Placeholder/_3}}]]


In [21]:
# sampled_train_dataset

In [22]:
# sampled_eval_dataset

### get user_id, item_id dictionary

In [23]:
users = ratings.map(lambda x: x["user_id"])
user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(
    mask_token=None)
user_ids_vocabulary.adapt(users.batch(1000))

2023-06-12 05:31:22.547297: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype double and shape [3000]
	 [[{{node Placeholder/_3}}]]


In [24]:
movies = movies.map(lambda x: x["movie_title"])
movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(
    mask_token=None)
movie_titles_vocabulary.adapt(movies.batch(1000))

2023-06-12 05:31:22.739105: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [137541]
	 [[{{node Placeholder/_0}}]]


In [25]:
movie_titles_vocabulary

Group by `user_id` to form lists for ranking models:


In [26]:
# for x in ds_train.take(1):
#   for key, value in x.items():
#     print(f"Shape of {key}: {value.shape}")
#     print(f"Example values of {key}: {value[:5].numpy()}")
#     print()

Generate batched features and labels:

In [27]:
ds_train = get_dataset(ratings=sampled_train_dataset, user_ids_vocabulary=user_ids_vocabulary)
ds_eval = get_dataset(ratings=sampled_eval_dataset, user_ids_vocabulary=user_ids_vocabulary)

Instructions for updating:
Use `tf.data.Dataset.ragged_batch` instead.


The `user_id` and `movie_title` tensors generated in `ds_train` are of shape `[32, None]`, where the second dimension is 100 in most cases except for the batches when less than 100 items grouped in lists. A model working on ragged tensors is thus used.

In [28]:
# ds_eval = eval_candidates

In [29]:
# ds_eval

In [30]:
for x, label in ds_train.take(1):
  for key, value in x.items():
    print(f"Shape of {key}: {value.shape}")
    print(f"Example values of {key}: {value[:3, :3].numpy()}")
    print()
  print(f"Shape of label: {label.shape}")
  print(f"Example values of label: {label[:3, :3].numpy()}")

Shape of user_id: (32, None)
Example values of user_id: [[b'3272766' b'3272766' b'3272766']
 [b'3272767' b'3272767' b'3272767']
 [b'3273022' b'3273022' b'3273022']]

Shape of movie_title: (32, None)
Example values of movie_title: [[b'B09C5VHGBZ' b'B09FZ8ZZB2' b'B09D3D8YR6']
 [b'B07WZZG98H' b'B01EA0NF8E' b'B00VYAWIRS']
 [b'B09L4BLSZR' b'B09NFHVKSG' b'B09J3LTNY7']]

Shape of next_item_normalized_weight: (32, None)
Example values of next_item_normalized_weight: [[0.42918456 0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]]

Shape of label: (32, None)
Example values of label: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


2023-06-12 05:31:28.639324: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2873]
	 [[{{node Placeholder/_0}}]]


In [31]:
for x, label in ds_eval.take(1):
  for key, value in x.items():
    print(f"Shape of {key}: {value.shape}")
    print(f"Example values of {key}: {value[:3, :3].numpy()}")
    print()
  print(f"Shape of label: {label.shape}")
  print(f"Example values of label: {label[:3, :3].numpy()}")

Shape of user_id: (2, None)
Example values of user_id: [[b'3273343' b'3273343' b'3273343']
 [b'3273343' b'3273343' b'3273343']]

Shape of movie_title: (2, None)
Example values of movie_title: [[b'B0068BBBIU' b'B000CST6NM' b'B09S6SXHS9']
 [b'B08MZK2T5Q' b'B09QT5CBBG' b'B094YJZY98']]

Shape of next_item_normalized_weight: (2, None)
Example values of next_item_normalized_weight: [[0. 0. 0.]
 [0. 0. 0.]]

Shape of label: (2, None)
Example values of label: [[0. 0. 0.]
 [0. 0. 0.]]


2023-06-12 05:31:29.594556: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [127]
	 [[{{node Placeholder/_0}}]]


## Define a model

Define a ranking model by inheriting from `tf.keras.Model` and implementing the `call` method:

In [35]:
# from abc import abstractmethod
# from tensorflow.keras import Model
# from typing import List



# class DLModel(Model):
#     def __init__(self):
#         super(DLModel, self).__init__()
#         self.layers_lst = []

#     @abstractmethod
#     def call(self, inputs, training=None, mask=None):
#         for layer in self.layers_lst:
#             inputs = layer(inputs)
#         return inputs

# class DNN(DLModel):
#     def __init__(self, hidden_dim: List[int], sigmoid=True) -> None:
#         super(DNN, self).__init__()
#         for dim in hidden_dim:
#             self.layers_lst.append(
#                 Dense(units=dim, use_bias=True)
#             )
#         self.sigmoid = sigmoid
#         if self.sigmoid:
#             self.layers_lst.append(
#                 Activation(activation="sigmoid")
#             )

In [36]:
feature_cols

['next_item_normalized_weight']

In [40]:
# class user_embedding(tf.keras.Model):
    

class MovieLensRankingModel(tf.keras.Model):

  def __init__(self, user_vocab, movie_vocab):
    super().__init__()

    # Set up user and movie vocabulary and embedding.
    self.user_vocab = user_vocab
    self.movie_vocab = movie_vocab
    self.user_embed = tf.keras.layers.Embedding(user_vocab.vocabulary_size(),
                                                63)
    self.movie_embed = tf.keras.layers.Embedding(movie_vocab.vocabulary_size(),
                                                 64)

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    # Define how the ranking scores are computed: 
    # Take the dot-product of the user embeddings with the movie embeddings.
    
    new_features = features['next_item_normalized_weight']

    user_embeddings = self.user_embed(self.user_vocab(features["user_id"]))
    
    user_embeddings = tf.concat([user_embeddings, new_features], 1)
    
    movie_embeddings = self.movie_embed(
        self.movie_vocab(features["movie_title"]))

    return tf.reduce_sum(user_embeddings * movie_embeddings, axis=2)

Create the model, and then compile it with ranking `tfr.keras.losses` and `tfr.keras.metrics`, which are the core of the TF-Ranking package. 

This example uses a ranking-specific **softmax loss**, which is a listwise loss introduced to promote all relevant items in the ranking list with better chances on top of the irrelevant ones. In contrast to the softmax loss in the multi-class classification problem, where only one class is positive and the rest are negative, the TF-Ranking library supports multiple relevant documents in a query list and non-binary relevance labels.

For ranking metrics, this example uses in specific **Normalized Discounted Cumulative Gain (NDCG)** and **Mean Reciprocal Rank (MRR)**, which calculate the user utility of a ranked query list with position discounts. For more details about ranking metrics, review evaluation measures [offline metrics](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Offline_metrics).

In [41]:
# def is_test(x, _):
#     return x % 4 == 0


# def is_train(x, y):
#     return not is_test(x, y)


# recover = lambda x, y: y

# # Split the dataset for training.
# test_dataset = ds_train.enumerate() \
#     .filter(is_test) \
#     .map(recover)

# # Split the dataset for testing/validation.
# train_dataset = ds_train.enumerate() \
#     .filter(is_train) \
#     .map(recover)

### Point-wise loss 

In [42]:


loss = tfr.keras.losses.MeanSquaredLoss(ragged=True)


model = MovieLensRankingModel(user_ids_vocabulary, movie_titles_vocabulary)
optimizer = tf.keras.optimizers.Adagrad(0.5)
eval_metrics = [
    # tfr.keras.metrics.get(key="ndcg", name="metric/ndcg", ragged=True),
    tfr.keras.metrics.get(key="mrr", name="metric/mrr", ragged=True)
]
model.compile(optimizer=optimizer, loss=loss, metrics=eval_metrics)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=5,
        verbose=1,
        restore_best_weights=True),
]
model.fit(
    ds_train,
    # train_dataset,
    epochs=epochs,
    validation_data=ds_eval,
    callbacks=callbacks
          # validation_split=0.1
)

Epoch 1/4


ValueError: in user code:

    File "/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/keras/utils/traceback_utils.py"

### Pair-wise loss 

In [ ]:
# # Create the ranking model, trained with a ranking loss and evaluated with
# # ranking metrics.

# loss = tfr.keras.losses.PairwiseHingeLoss(ragged=True)


# model = MovieLensRankingModel(user_ids_vocabulary, movie_titles_vocabulary)
# optimizer = tf.keras.optimizers.Adagrad(0.5)
# eval_metrics = [
#     tfr.keras.metrics.get(key="ndcg", name="metric/ndcg", ragged=True),
#     tfr.keras.metrics.get(key="mrr", name="metric/mrr", ragged=True)
# ]
# model.compile(optimizer=optimizer, loss=loss, metrics=eval_metrics)
# model.fit(ds_train, epochs=3)

### List-wise loss 

In [ ]:
# # Create the ranking model, trained with a ranking loss and evaluated with
# # ranking metrics.

# loss = tfr.keras.losses.get(
#     loss=tfr.keras.losses.RankingLossKey.SOFTMAX_LOSS, ragged=True)

# model = MovieLensRankingModel(user_ids_vocabulary, movie_titles_vocabulary)
# optimizer = tf.keras.optimizers.Adagrad(0.5)
# eval_metrics = [
#     tfr.keras.metrics.get(key="ndcg", name="metric/ndcg", ragged=True),
#     tfr.keras.metrics.get(key="mrr", name="metric/mrr", ragged=True)
# ]
# model.compile(optimizer=optimizer, loss=loss, metrics=eval_metrics)
# model.fit(ds_train, epochs=3)

Generate predictions and evaluate.

## Model eval function 

In [ ]:
def rank_rec(candidate_pl, model, topn=100, ground_truth_included=False):
    candidate_pl = candidate_pl.collect().to_pandas()
    print(f"candidate pl shape: {candidate_pl.shape}")

    user_tensor = tf.reshape(tf.convert_to_tensor(candidate_pl[user_col]), (1, -1))
    item_tensor = tf.reshape(tf.convert_to_tensor(candidate_pl[item_col]), (1, -1))
    new_inputs = {
        "user_id":user_tensor
            ,
        "movie_title":item_tensor
    }
    output = model(new_inputs)

    candidate_pl['predict'] = tf.squeeze(output, axis=0).numpy()


    test_result = (
        pl.from_pandas(candidate_pl)
         .lazy()
         # .with_columns(
         #     pl.col(item_col).alias('next_item_prediction')
         #     , pl.col(user_col).alias('session_id').cast(pl.Int64)
         # )
         .with_columns(
            pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
         )
         .sort(['session_id', 'rank'])
         .filter(pl.col('rank')<=topn)
    )
    if ground_truth_included:
        test_result = (
            test_result         
            .groupby(['session_id'])
             .agg(
                 pl.col('next_item_prediction')
                 , pl.col('next_item').unique()#.arr.get(0).cast(pl.Utf8)
             ).with_columns(
                 pl.col('next_item').arr.get(0).alias('next_item')
                 , pl.col(user_col).cast(pl.Int64)
             )
        )
    else:
        test_result = (
            test_result         
            .groupby(['session_id'])
             .agg(
                 pl.col('next_item_prediction')
             ).with_columns(
                 pl.col(user_col).cast(pl.Int64)
             )
        )
    return test_result


## Model Eval 

In [ ]:
ranker_eval_pl = rank_rec(candidate_pl=eval_candidates
                        , model=model
                          , topn=topn
                          , ground_truth_included=True
                         )

In [ ]:
ranker_eval_pl.head().collect()

In [ ]:
model_eval(eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))


## Test data 

In [139]:
test_result = rank_rec(candidate_pl=test_candidates
                        , model=model
                          , topn=topn
                           , ground_truth_included=False
                         )

candidate pl shape: (3000, 56)


In [140]:
test_result.head().collect()

session_id,next_item_prediction
i64,list[str]
4331307,"[""B0B56NWXYH"", ""B09Z1VH85Q"", … ""B09N7R5P99""]"
4331319,"[""B09Z1VH85Q"", ""B084J38SRL"", … ""B019GJLER8""]"
4331312,"[""B011EI9PKW"", ""B09WMK1RP4"", … ""B07Y7YLB7Z""]"
4331311,"[""B09XP9BXV2"", ""B08T9RDVMT"", … ""B09MSB4X2G""]"
4331313,"[""B008SO7JZ4"", ""B09FKWS793"", … ""B07S3BJ294""]"


In [141]:

predictions = test_pl.join(test_result, how='left', on='session_id').collect()[['locale', 'next_item_prediction']].to_pandas()

In [142]:
predictions_pl = (
    test_pl.join(test_result, how='left', on='session_id')
        .with_columns(
            pl.when(pl.col('next_item_prediction').is_null())
                .then(pl.col('prev_items'))
                .otherwise(pl.col('next_item_prediction')).alias('next_item_prediction')
        )
).collect()

predictions = (predictions_pl[['locale', 'next_item_prediction']].to_pandas())
prediction_summary = (
    predictions_pl
        .explode('next_item_prediction')
        .with_columns(
            pl.col('next_item_prediction').is_null().alias('whether_null')
        )
        .groupby('session_id')
        .agg(
            pl.col('whether_null').sum().alias('null_num')
            , pl.count().alias('rec_num')
        )
)
assert (
    prediction_summary.filter(pl.col('null_num')>0)
).shape[0] == 0


check_predictions(predictions, test_sessions=test_pl.collect().to_pandas(), 
                  # check_products=True, product_df=products
                 )
# Its important that the parquet file you submit is saved with pyarrow backend

assert (
    pl.from_pandas(predictions)
        .explode('next_item_prediction')
        .filter(
            pl.col('next_item_prediction').is_null()
                     )
).shape[0] == 0
assert predictions.shape[0] == test_pl.collect().shape[0]
if not debug:
    predictions.to_parquet(sub_file, engine='pyarrow')

In [143]:
! ls -al {sub_file}

-rw-r--r-- 1 jupyter jupyter 28918656 Jun 11 06:31 ../data/sub_files/task2_rank_tf_v3.parque


In [ ]:
# !aicrowd submission create -c task-2-next-product-recommendation-for-underrepresented-languages -f {sub_file}

# Ranking pipeline example 

Refer to [github page](https://github.com/tensorflow/ranking/blob/master/tensorflow_ranking/examples/tf_ranking_canned_dnn.py) for more details 

In [16]:
# # Copyright 2022 The TensorFlow Ranking Authors.
# #
# # Licensed under the Apache License, Version 2.0 (the "License");
# # you may not use this file except in compliance with the License.
# # You may obtain a copy of the License at
# #
# #     http://www.apache.org/licenses/LICENSE-2.0
# #
# # Unless required by applicable law or agreed to in writing, software
# # distributed under the License is distributed on an "AS IS" BASIS,
# # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# # See the License for the specific language governing permissions and
# # limitations under the License.

# r"""TF-Ranking example code for training a canned DNN estimator.

# The supported proto formats are listed at ../python/data.py.
# --------------------------------------------------------------------------------
# Sample command lines:

# MODEL_DIR=/tmp/output && \
# TRAIN=tensorflow_ranking/examples/data/train_numerical_elwc.tfrecord && \
# EVAL=tensorflow_ranking/examples/data/vali_numerical_elwc.tfrecord && \
# rm -rf $MODEL_DIR && \
# bazel build -c opt \
# tensorflow_ranking/examples/tf_ranking_canned_dnn_py_binary && \
# ./bazel-bin/tensorflow_ranking/examples/tf_ranking_canned_dnn_py_binary \
# --train_input_pattern=$TRAIN \
# --eval_input_pattern=$EVAL \
# --model_dir=$MODEL_DIR

# You can use TensorBoard to display the training results stored in $MODEL_DIR.

# Notes:
#   * Use --alsologtostderr if the output is not printed into screen.
# """

# from absl import flags

# import tensorflow as tf
# import tensorflow_ranking as tfr

# flags.DEFINE_string("train_input_pattern", None,
#                     "Input file path used for training.")
# flags.DEFINE_string("eval_input_pattern", None,
#                     "Input file path used for eval.")
# flags.DEFINE_string("model_dir", None, "Output directory for models.")
# flags.DEFINE_integer("batch_size", 32, "The batch size for train.")
# flags.DEFINE_integer("num_train_steps", 15000, "Number of steps for train.")
# flags.DEFINE_integer("num_eval_steps", 10, "Number of steps for evaluation.")
# flags.DEFINE_integer("checkpoint_secs", 30,
#                      "Saves a model checkpoint every checkpoint_secs seconds.")
# flags.DEFINE_integer("num_checkpoints", 100,
#                      "Saves at most num_checkpoints checkpoints in workspace.")
# flags.DEFINE_integer("num_features", 136, "Number of features per example.")
# flags.DEFINE_integer(
#     "list_size", 100,
#     "List size used for training. Use None for dynamic list size.")
# flags.DEFINE_float("learning_rate", 0.05, "Learning rate for optimizer.")
# flags.DEFINE_float("dropout", 0.8, "The dropout rate before output layer.")
# flags.DEFINE_list("hidden_layer_dims", ["64", "32", "16"],
#                   "Sizes for hidden layers.")
# flags.DEFINE_string("loss", "approx_ndcg_loss",
#                     "The RankingLossKey for the loss function.")
# flags.DEFINE_bool("convert_labels_to_binary", False,
#                   "If true, relevance labels are set to either 0 or 1.")
# flags.DEFINE_bool("listwise_inference", False,
#                   "If true, exports accept `data_format` while serving.")

# FLAGS = flags.FLAGS

# _LABEL_FEATURE = "utility"


# def context_feature_columns():
#   """Returns context feature columns."""
#   return {}


# def example_feature_columns():
#   """Returns the example feature columns."""
#   feature_names = [
#       "custom_features_{}".format(i + 1) for i in range(FLAGS.num_features)
#   ]
#   return {
#       name:
#       tf.feature_column.numeric_column(name, shape=(1,), default_value=0.0)
#       for name in feature_names
#   }


# def train_and_eval():
#   """Train and Evaluate."""
#   optimizer = tf.compat.v1.train.AdagradOptimizer(
#       learning_rate=FLAGS.learning_rate)

#   estimator = tfr.estimator.make_dnn_ranking_estimator(
#       example_feature_columns(),
#       FLAGS.hidden_layer_dims,
#       context_feature_columns=context_feature_columns(),
#       optimizer=optimizer,
#       learning_rate=FLAGS.learning_rate,
#       loss=FLAGS.loss,
#       loss_reduction=tf.compat.v1.losses.Reduction.SUM_OVER_BATCH_SIZE,
#       activation_fn=tf.nn.relu,
#       dropout=FLAGS.dropout,
#       use_batch_norm=True,
#       model_dir=FLAGS.model_dir)

#   hparams = {"train_input_pattern": FLAGS.train_input_pattern,
#              "eval_input_pattern": FLAGS.eval_input_pattern,
#              "learning_rate": FLAGS.learning_rate,
#              "train_batch_size": FLAGS.batch_size,
#              "eval_batch_size": FLAGS.batch_size,
#              "predict_batch_size": FLAGS.batch_size,
#              "num_train_steps": FLAGS.num_train_steps,
#              "num_eval_steps": FLAGS.num_eval_steps,
#              "checkpoint_secs": FLAGS.checkpoint_secs,
#              "num_checkpoints": FLAGS.num_checkpoints,
#              "loss": FLAGS.loss,
#              "list_size": FLAGS.list_size,
#              "convert_labels_to_binary": FLAGS.convert_labels_to_binary,
#              "listwise_inference": FLAGS.listwise_inference,
#              "model_dir": FLAGS.model_dir}

#   ranking_pipeline = tfr.ext.pipeline.RankingPipeline(
#       context_feature_columns(),
#       example_feature_columns(),
#       hparams,
#       estimator=estimator,
#       label_feature_name=_LABEL_FEATURE,
#       label_feature_type=tf.int64)

#   ranking_pipeline.train_and_eval()


# def main(_):
#   tf.compat.v1.set_random_seed(1234)
#   tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)
#   train_and_eval()


# if __name__ == "__main__":
#   flags.mark_flag_as_required("train_input_pattern")
#   flags.mark_flag_as_required("eval_input_pattern")
#   flags.mark_flag_as_required("model_dir")

#   tf.compat.v1.app.run()

In [17]:
# # Copyright 2022 The TensorFlow Ranking Authors.
# #
# # Licensed under the Apache License, Version 2.0 (the "License");
# # you may not use this file except in compliance with the License.
# # You may obtain a copy of the License at
# #
# #     http://www.apache.org/licenses/LICENSE-2.0
# #
# # Unless required by applicable law or agreed to in writing, software
# # distributed under the License is distributed on an "AS IS" BASIS,
# # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# # See the License for the specific language governing permissions and
# # limitations under the License.

# """Tests for tf_ranking_canned_dnn.py."""

# import os

# from absl import flags
# from absl.testing import flagsaver
# from absl.testing import parameterized

# import tensorflow as tf

# from google.protobuf import text_format
# from tensorflow_ranking.examples import tf_ranking_canned_dnn
# from tensorflow_serving.apis import input_pb2

# FLAGS = flags.FLAGS

# ELWC = text_format.Parse(
#     """
#     context {
#     }
#     examples {
#       features {
#         feature {
#           key: "custom_features_1"
#           value { float_list { value: 1.0 } }
#         }
#         feature {
#           key: "custom_features_2"
#           value { float_list { value: 1.5 } }
#         }
#         feature {
#           key: "utility"
#           value { int64_list { value: 1 } }
#         }
#       }
#     }
#     examples {
#       features {
#         feature {
#           key: "custom_features_1"
#           value { float_list { value: 1.0 } }
#         }
#         feature {
#           key: "custom_features_3"
#           value { float_list { value: 2.1 } }
#         }
#         feature {
#           key: "utility"
#           value { int64_list { value: 0 } }
#         }
#       }
#     }""", input_pb2.ExampleListWithContext())


# def _write_tfrecord_files(path):
#   elwc_list = [ELWC.SerializeToString()] * 10
#   if tf.io.gfile.exists(path):
#     tf.io.gfile.remove(path)

#   with tf.io.TFRecordWriter(path) as writer:
#     for elwc in elwc_list:
#       writer.write(elwc)


# class TFRankingCannedDNNTest(tf.test.TestCase, parameterized.TestCase):

#   def setUp(self):
#     super(TFRankingCannedDNNTest, self).setUp()
#     tf.compat.v1.reset_default_graph()

#     # Prepares model directory, and train and eval data.
#     self._base_model_dir = tf.compat.v1.test.get_temp_dir() + "/model/"
#     tf.io.gfile.makedirs(self._base_model_dir)
#     self._data_file = os.path.join(self._base_model_dir, "elwc.tfrecord")
#     _write_tfrecord_files(self._data_file)

#   def tearDown(self):
#     super(TFRankingCannedDNNTest, self).tearDown()
#     if self._base_model_dir:
#       tf.io.gfile.rmtree(self._base_model_dir)
#     self._base_model_dir = None

#   @parameterized.named_parameters(("enable_listwise_inference", True),
#                                   ("disable_listwise_inference", False))
#   def test_train_and_eval(self, listwise_inference):
#     self._model_dir = self._base_model_dir + "/" + str(listwise_inference)
#     with flagsaver.flagsaver(
#         train_input_pattern=self._data_file,
#         eval_input_pattern=self._data_file,
#         model_dir=self._model_dir,
#         num_features=3,
#         num_train_steps=10,
#         listwise_inference=listwise_inference):
#       tf_ranking_canned_dnn.train_and_eval()


# if __name__ == "__main__":
#   tf.test.main()